### 6월 26일

## OpenAI 이어서

### Web search
- 모델이 응답을 생성하기 전에 웹에서 최신 정보를 검색할 수 있도록 허용
- tool_choice 매개변수를 사용하여 web_search_preview 도구의 사용을 강제할 수 있음
    - 이를 {type: "web_search_preview"}로 설정하면 지연 시간을 줄이고 더 일관된 결과를 얻을 수 있음

- 출력 및 인용
    - 웹 검색 도구를 사용하는 모델 응답은 두 부분으로 구성
        - 웹 검색 호출의 ID를 포함하는 web_search_call 출력 항목.
        - 출력 항목에 포함되는 내용
            - 메시지 콘텐츠의 텍스트 결과(message.content[0].text)
            - 인용된 URL에 대한 주석(message.content[0].annotations)
    - 기본적으로 모델의 응답에는 웹 검색 결과에서 발견된 URL에 대한 인라인 인용이 포함
    - url_citation 주석 객체에는 인용된 소스의 URL, 제목 및 위치가 포함

### User location
- 지리적 위치에 따라 검색 결과를 필터링하려면 국가, 도시, 지역 및/또는 시간대를 사용하여 대략적인 사용자 위치를 지정

- 도시 및 지역 필드는 자유 텍스트 문자열로, 각각 Minneapolis와 Minnesota와 같음

- 국가 필드는 두 글자 ISO 국가 코드로, 예를 들어 US와 같음, KR

- 시간대 필드는 IANA 시간대 코드로, 예를 들어 America/Chicago와 같음, Asia/Seoul


In [ ]:
#예시 코드
tools=[{
    "type": "web_search_preview",
    "user_location": {
        "type": "approximate",
        "country": "GB",
        "city": "London",
        "region": "London",
    }
}],


tools = [{
    "type": "web_search_preview",
    "user_location": {
        "type": "approximate",
        "country": "KR", # ISO 2-letter 국가 코드
        "city": "Seoul",
        "region": "Seoul",
        "timezone": "Asia/Seoul" # IANA 표준 시간대 코드
    }
}]

### 검색 컨텍스트 크기
- 이 도구를 사용할 때 search_context_size 매개변수는 웹에서 검색하여 도구가 응답을 구성하는 데 사용하는 컨텍스트의 양을 제어

- 검색 도구가 사용하는 토큰은 응답 생성 요청에서 모델 매개변수로 지정된 주요 모델의 컨텍스트 창에 영향을 미치지 않음

- 이러한 토큰은 한 번의 대화 단계에서 다음 단계로 전달되지 않음

- 단순히 도구의 응답을 구성하는 데 사용된 후 버려짐

- **컨텍스트 크기가 작으면 비용을 아낄 수 있지만, 품질이 낮아질 수 있다!**
    - 컨텍스트 크기를 선택
        - 비용: 검색 도구 가격은 이 매개변수의 값에 따라 달라짐. 더 큰 컨텍스트 크기는 더 높은 비용이 발생
        - 품질: 더 큰 검색 컨텍스트 크기는 일반적으로 더 풍부한 컨텍스트를 제공하여 더 정확하고 포괄적인 답변을 생성
        - 지연 시간: 더 큰 컨텍스트 크기는 더 많은 토큰을 처리해야 하므로 도구 응답 시간이 느려질 수 있음

    - 크기 옵션
        - high: 가장 포괄적인 컨텍스트, 가장 높은 비용, 가장 느린 응답.
        - medium(기본): 컨텍스트, 비용, 지연 시간의 균형 잡힌 조합.
        - low: 가장 적은 컨텍스트, 가장 낮은 비용, 가장 빠른 응답이지만 답변 품질이 낮을 수 있음

    - 검색 도구가 사용하는 토큰은 주요 모델의 토큰 사용량에 영향을 주지 않으며, 턴 간에 전달되지 않음

In [ ]:
response = client.responses.create(
    model="gpt-4.1",
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "low",
    }],
    input="What movie won best picture in 2025?",
)
print(response.output_text)

### File search
- **파일을 등록해두고 파일 데이터를 바탕으로 검색, 검색 결과를 기반으로 응답을 만들어주는 것이다!**(지식 기반의 정보 검색)

- 모델이 응답을 생성하기 전에 파일에서 관련 정보를 검색할 수 있도록 허용

- 파일 검색은 Responses API에서 사용할 수 있는 도구

- 이 도구는 모델이 이전에 **업로드된 파일의 지식 기반**에서 의미론적 및 키워드 검색을 통해 정보를 검색

- **벡터 저장소**를 생성하고 **파일을 업로드**함으로써, 모델의 내재된 지식을 확장하기 위해 이러한 지식 기반이나 벡터 저장소에 접근할 수 있음
    - 저장소를 만들고 response가 참조할 수 있도록 등록을 해준 뒤에 파일을 업로드해주어야 한다!

- 이 도구는 OpenAI에서 호스팅 및 관리되는 도구이므로, 실행을 처리하기 위해 코드를 구현할 필요가 없음

- 모델이 이 도구를 사용하기로 결정하면 자동으로 도구를 호출하여 파일에서 정보를 검색하고 출력을 반환

- 일반적으로 http와 같은 경로는 공개적인 정보로 판단하자

In [ ]:
#ector store 생성
vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

### 정리 및 UI
- 유저 친화적인 UI가 있으면 사용자 입장에서 접근성이 높아진다
- 파이썬과 직접 연결하여 처리할 수 있는 Streamlit을 사용하여 간단한 챗봇 만들기

- 모델학습 및 모델 디자인 --> **분석**,
- 함수를 동애, openAPI 사용, function calling, 입력 -> 출력 작업을 **자동화**로 생각할 수 있다.

- 웹 서치는 설정만 하면 되지만, 파일 서치는 추가로 만들어주어야 하는 단계들이 존재한다!

### Frontend
- 프론트엔드 서버: 사용자와 웹 애플리케이션의 인터페이스 역할
    - 웹사이트나 웹 애플리케이션의 사용자 경험을 담당하는 서버
    - 주로 사용자가 웹 브라우저를 통해 접속하는 화면이나 UI를 제공
    - 사용자의 요청을 백엔드 서버로 전달하는 중요한 중개 역할

- Streamlit
    - Streamlit은 서버처럼 구동을 시켜주는 프레임워크
    - 프론트엔트 --> user들과 접점, 이 접점을 만들어주는 것이 Streamlit
    - CSS, HTML, JS 등을 사용하지 않는 프레임워크. 실제 웹 개발에는 사용하기 힘들다. (**완벽한 서버의 역할은 X**)